# scheduling



### moving purple events



#### the code

how to space out purple event?

assuming I can only get 1 task done per day


In [ ]:
var importer = require('../Core')
var {google} = require('googleapis')
var util = require('util')
var {
    correctCalendarId, authorizeCalendar, getDaysEvents, d3Heatmap
} = importer.import([
    'lookup calendar name', 'authorize google calendar', 'days events', 'd3 calendar'
])

var DAYS = 1200 // plus or minus
var SIMUL = 3
var options = {
}
          
var now = new Date()
// TODO: batch promises?
function batchPromises(options) {
    const promises = Array.from(Array(Math.ceil(DAYS / SIMUL)).keys()).map((acc, i) => {
        const daysBeforeAfter = i - Math.ceil(DAYS / SIMUL / 2)
        return resolve => Promise.all(Array.from(Array(SIMUL).keys()).map(day => {
            const date = new Date(now)
            date.setDate(date.getDate() + daysBeforeAfter * SIMUL + day)
            return getDaysEvents(date, options)
        }))
            .then(r => {
                setTimeout(() => resolve(r), 100)
            })
    });
    return importer.runAllPromises(promises)
        .then(r => [].concat.apply([], r))
}

function moveEvent(e, options) {
    var events
    var newTime = new Date(now)
    newTime.setDate(newTime.getDate() + i)
    newTime.setHours(12, 0, 0)
    return authorizeCalendar(options)
        .then(calendar => events = calendar.events)
        .then(() => util.promisify(events.patch.bind(events))({
            eventId: e.event.id,
            calendarId: options.calendarId,
            auth: options.auth,
            resource: {
                start: { dateTime: new Date(newTime.getTime()) },
                end: { dateTime: new Date(newTime.getTime() + 120 * 60 * 1000) },
            }
        }))
        .then(r => new Promise(resolve => {
            setTimeout(() => resolve(r), 100);
        }))
    // TODO: skip dates using chrono description relative to each date
    // TODO: skip overlap using study sauce algorithm
    // TODO: skip boundaries using a new algorithm for defining inside and outside times
}

function adjustPurpleEvents(options) {
    if(!options) options = {}
    return Promise.resolve()
        .then(() => correctCalendarId(options))
        .then(() => typeof global.listEvents === 'undefined'
              ? batchPromises(options)
              : Promise.resolve(global.listEvents))
        .then(r => {
            global.listEvents = r
            const purpleEvents = r.filter(e => e.event.colorId === '1')
            const promises = purpleEvents.map((e, i) => moveEvent(e, options))
            return importer.runAllPromises()
        })
        .then(r => d3Heatmap(r.map((e, i) => ({
            id: e.id,
            start: new Date(e.start.dateTime),
            end: new Date(e.end.dateTime),
        }))))
}

module.exports = adjustPurpleEvents


### study sauce scheduling



how to schedule study time?

how to schedule hours per event?

generalize from study sauce v2


how to schedule inside outside of time bounds?



how to remove overlaps?

how to remove overlaps from events with alloable wiggle room?



how to prioritize events based on glob summary?




calendar search heatmap?



In [ ]:
var importer = require('../Core');
var {
    getOauthClient,
    d3Heatmap,
    listEvents,
    correctCalendarId,
    sumEvents
} = importer.import([
    'import google calendar api',
    'd3 calendar',
    'list events',
    'lookup calendar name',
    'sum a list of events'
]);

var options = {
    
}

function calendarHeatmap(search, calendar) {
    if(calendar) {
        options.calendarId = calendar;
    }
    var total = 0;
    return getOauthClient(options)
        .then(() => correctCalendarId(options))
        .then(() => importer.runAllPromises(search.split('|').map(term => resolve => {
            return listEvents({
                auth: options.auth,
                calendarId: options.calendarId,
                q: term
            }).then(r => {
                console.log(term);
                console.log(r.map(e => e.event.summary).slice(0, 10));
                const hours = sumEvents(r);
                console.log(hours + ' hrs');
                total += hours;
                return resolve(r);
            })
        })))
        .then(r => [].concat(...r.map((e, i) => ({
            id: e.id,
            start: new Date(e.event.start.dateTime),
            end: new Date(e.event.end.dateTime),
        }))))
        .then(r => {
            console.log(total + ' hrs total');
            console.log((total / 41) + ' hrs / week') // subtract from 48 weeks, 2 weeks of mental health, 2 weeks of holidays
            return [d3Heatmap(r)]
        })
}
module.exports = calendarHeatmap;

if(typeof $$ !== 'undefined') {
    $$.async();
//    calendarHeatmap('work')
    calendarHeatmap('portal|eloqua|zuora|angular|webpack|renewals|unit test|unit tests|unit testing|selenium|angular2|pricing|jupytangular|notebook|work|sos|study sauce|notebooks|jupyter|cloud|docker|build')
    /*
    5776.5 hrs total
    120.34375 hrs / week
    */
//    calendarHeatmap('portal|eloqua|zuora|angular|webpack|renewals|unit test|unit tests|unit testing|selenium|angular2|pricing')
//    calendarHeatmap('portal')
    /*
    1353.75 hrs total
    28.203125 hrs / week
    */
        .then(r => $$.html(r[0]))
        .catch(e => $$.sendError(e))
}


search calendar?

In [ ]:
var importer = require('../Core');
var {
    getOauthClient,
    listEvents,
    correctCalendarId,
} = importer.import([
    'import google calendar api',
    'list events',
    'lookup calendar name',
]);

var options = {
    
}

function searchCalendar(search, calendar) {
    if(calendar) {
        options.calendarId = calendar;
    }
    var total = 0;
    return getOauthClient(options)
        .then(() => correctCalendarId(options))
        .then(() => importer.runAllPromises(search.split('|').map(term => resolve => {
            return listEvents({
                auth: options.auth,
                calendarId: options.calendarId,
                q: term
            }).then(r => {
                console.log(term);
                console.log(r.map(e => e.event.summary).slice(0, 10));
                return resolve(r);
            })
        })))
}
module.exports = searchCalendar;

if(typeof $$ !== 'undefined') {
    $$.async();
    searchCalendar('blur')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}
